Capstone Project – The Battle of Neighborhoods

1. Installing and Importing Python Libraries and Dependencies

In [1]:
!pip install geocoder
!pip install folium

     |████████████████████████████████| 102kB 8.3MB/s ta 0:00:011
     |████████████████████████████████| 102kB 7.9MB/s ta 0:00:011


In [2]:
import numpy as np

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values # to get coordinates

import requests
import geocoder# library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries  are imported.")

Libraries  are imported.


2. Scrap data from Wikipedia page into a DataFrame

In [3]:
# send the GET request
data = requests.get("https://commons.wikimedia.org/wiki/Category:Suburbs_of_Hyderabad,_India").text

In [4]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [5]:

# create a list to store neighborhood data
neighborhoodList = []

In [6]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [7]:
# create a new DataFrame from the list
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})
kl_df.head()

,Neighborhood
0,"► Abids‎ (1 C, 13 F)"
1,"► Alwal‎ (1 C, 1 F)"
2,"► Ameerpet, Hyderabad‎ (3 C, 21 F)"
3,"► Bandlaguda, Rangareddy‎ (1 C, 2 F)"
4,"► Banjara Hills‎ (3 C, 25 F)"


In [8]:
kl_df.shape

(54, 1)

3. Get the geographical coordinates

In [9]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Hyderabad, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [10]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]

In [11]:
coords

[[17.389800000000037, 78.47658000000007],
 [17.535430000000076, 78.54427000000004],
 [17.43482000000006, 78.44949000000008],
 [17.299820000000068, 78.46495000000004],
 [17.415350000000046, 78.43435000000005],
 [17.40211000000005, 78.47770000000008],
 [17.447290000000066, 78.45396000000005],
 [17.40954000000005, 78.57896000000005],
 [17.536218869427803, 78.2350425425703],
 [17.40893503530367, 78.32674007784891],
 [17.40301000000005, 78.49792000000008],
 [17.40893503530367, 78.32674007784891],
 [17.368570000000034, 78.53515000000004],
 [17.409950000000038, 78.48229000000003],
 [17.45333000000005, 78.43034000000006],
 [17.43181000000004, 78.38636000000008],
 [17.522760000000062, 78.43862000000007],
 [17.46686941076456, 78.24915353871232],
 [17.389410000000055, 78.40406000000007],
 [17.32707000000005, 78.60533000000004],
 [17.448230000000024, 78.37429000000003],
 [17.399230000000045, 78.48073000000005],
 [17.36838000000006, 78.39999000000006],
 [17.42865000000006, 78.39762000000007],
 [17.

In [12]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [13]:
# merge the coordinates into the original dataframe
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']

In [14]:

# check the neighborhoods and the coordinates
print(kl_df.shape)
kl_df

(54, 3)


,Neighborhood,Latitude,Longitude
0,"► Abids‎ (1 C, 13 F)",17.389800,78.476580
1,"► Alwal‎ (1 C, 1 F)",17.535430,78.544270
2,"► Ameerpet, Hyderabad‎ (3 C, 21 F)",17.434820,78.449490
3,"► Bandlaguda, Rangareddy‎ (1 C, 2 F)",17.299820,78.464950
4,"► Banjara Hills‎ (3 C, 25 F)",17.415350,78.434350
5,"► Basheerbagh‎ (1 C, 7 F)",17.402110,78.477700
6,"► Begumpet‎ (5 C, 9 F)",17.447290,78.453960
7,► Boduppal‎ (2 F),17.409540,78.578960
8,"► Bolarum‎ (3 C, 1 F)",17.536219,78.235043
9,"► Cavalry Barracks, Hyderabad‎ (1 C)",17.408935,78.326740


In [15]:
kl_df.to_csv("kl_df.csv", index=False)


4. Create a map of Hyderabad with neighborhoods superimposed on top

In [16]:
# get the coordinates of Hyderabad
address = 'Hyderabad, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Hyderabad, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Hyderabad, India 17.3616079, 78.4746286.


In [17]:
# create map of Hyderabad using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [18]:
# save the map as HTML file
map_kl.save('map_kl.html')

5. Use the Foursquare API to explore the neighborhoods

In [19]:
# define Foursquare Credentials and Version
CLIENT_ID = '1PEFBPC43L0KTSOSEW3RBCW1S5TZDYV0ZU1O5BQKGXUADTSI' # your Foursquare ID
CLIENT_SECRET = '41GRCDOR4GDVILETGZRA45C1ZZTHE3JPRNWLTMKZF0VQGWD2' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1PEFBPC43L0KTSOSEW3RBCW1S5TZDYV0ZU1O5BQKGXUADTSI
CLIENT_SECRET:41GRCDOR4GDVILETGZRA45C1ZZTHE3JPRNWLTMKZF0VQGWD2


Now, let's get the top 100 venues that are within a radius of 2000 meters.

In [20]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [21]:

# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(2168, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,"► Abids‎ (1 C, 13 F)",17.3898,78.47658,Santosh Dhaba,17.388485,78.479509,Indian Restaurant
1,"► Abids‎ (1 C, 13 F)",17.3898,78.47658,Pragati,17.388088,78.481134,South Indian Restaurant
2,"► Abids‎ (1 C, 13 F)",17.3898,78.47658,Mayur Pan Shop,17.388894,78.480578,Juice Bar
3,"► Abids‎ (1 C, 13 F)",17.3898,78.47658,Karachi Bakery,17.383454,78.475075,Bakery
4,"► Abids‎ (1 C, 13 F)",17.3898,78.47658,Ram ki Bandi,17.382398,78.475014,Food Truck


Let's check how many venues were returned for each neighorhood

In [22]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
"► Abids‎ (1 C, 13 F)",78,78,78,78,78,78
"► Alwal‎ (1 C, 1 F)",4,4,4,4,4,4
"► Ameerpet, Hyderabad‎ (3 C, 21 F)",100,100,100,100,100,100
"► Bandlaguda, Rangareddy‎ (1 C, 2 F)",4,4,4,4,4,4
"► Banjara Hills‎ (3 C, 25 F)",100,100,100,100,100,100
"► Basheerbagh‎ (1 C, 7 F)",96,96,96,96,96,96
"► Begumpet‎ (5 C, 9 F)",53,53,53,53,53,53
► Boduppal‎ (2 F),6,6,6,6,6,6
"► Bolarum‎ (3 C, 1 F)",2,2,2,2,2,2



Let's find out how many unique categories can be curated from all the returned venues


In [23]:

print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 146 uniques categories.


In [24]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Indian Restaurant', 'South Indian Restaurant', 'Juice Bar',
       'Bakery', 'Food Truck', 'Hotel', 'Ice Cream Shop', 'Shoe Store',
       'Diner', 'Chaat Place', 'Neighborhood', 'Lounge', 'Burger Joint',
       'Dessert Shop', 'Café', 'Stadium', 'Snack Place', 'Science Museum',
       'Chinese Restaurant', 'Mobile Phone Shop', 'Restaurant',
       'Smoke Shop', 'Coffee Shop', 'Fast Food Restaurant',
       'Breakfast Spot', 'Hotel Bar', 'Department Store', 'Bar',
       'Shopping Mall', 'Multiplex', 'Food', 'Performing Arts Venue',
       'Gaming Cafe', 'Indie Movie Theater', 'Farmers Market',
       'Pizza Place', 'Fried Chicken Joint', 'Hookah Bar',
       'Clothing Store', 'Sandwich Place', 'Food Court', 'Pharmacy',
       'Bus Station', 'Golf Course', 'Asian Restaurant', 'ATM', 'Pub',
       'Bookstore', 'American Restaurant',
       'Vegetarian / Vegan Restaurant'], dtype=object)

In [25]:

# check if the results contain "Restauranrt"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

6. Analyze Neighborhood

In [26]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(2168, 147)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Service,American Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,BBQ Joint,Bakery,Bar,Bed & Breakfast,Beer Garden,Bengali Restaurant,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Burger Joint,Bus Station,Cable Car,Cafeteria,Café,Chaat Place,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Dumpling Restaurant,Duty-free Shop,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Historic Site,History Museum,Hookah Bar,Hotel,Hotel Bar,Hotel Pool,Hunan Restaurant,Hyderabadi Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irish Pub,Italian Restaurant,Jewelry Store,Juice Bar,Lake,Light Rail Station,Liquor Store,Lounge,Mattress Store,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Neighborhood,New American Restaurant,Nightclub,North Indian Restaurant,Office,Outdoors & Recreation,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Pub,Racetrack,Rajasthani Restaurant,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Social Club,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Taxi Stand,Tea Room,Temple,Thai Restaurant,Train Station,Vegetarian / Vegan Restaurant,Volleyball Court
0,"► Abids‎ (1 C, 13 F)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"► Abids‎ (1 C, 13 F)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,"► Abids‎ (1 C, 13 F)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"► Abids‎ (1 C, 13 F)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"► Abids‎ (1 C, 13 F)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [27]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped


(52, 147)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Service,American Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,BBQ Joint,Bakery,Bar,Bed & Breakfast,Beer Garden,Bengali Restaurant,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Burger Joint,Bus Station,Cable Car,Cafeteria,Café,Chaat Place,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Dumpling Restaurant,Duty-free Shop,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Historic Site,History Museum,Hookah Bar,Hotel,Hotel Bar,Hotel Pool,Hunan Restaurant,Hyderabadi Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irish Pub,Italian Restaurant,Jewelry Store,Juice Bar,Lake,Light Rail Station,Liquor Store,Lounge,Mattress Store,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Neighborhood,New American Restaurant,Nightclub,North Indian Restaurant,Office,Outdoors & Recreation,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Pub,Racetrack,Rajasthani Restaurant,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Social Club,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Taxi Stand,Tea Room,Temple,Thai Restaurant,Train Station,Vegetarian / Vegan Restaurant,Volleyball Court
0,"► Abids‎ (1 C, 13 F)",0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.038462,0.012821,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.012821,0.00,0.012821,0.012821,0.000000,0.000000,0.038462,0.012821,0.051282,0.012821,0.00,0.025641,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.025641,0.038462,0.012821,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.012821,0.051282,0.000000,0.012821,0.000000,0.012821,0.000000,0.012821,0.00,0.012821,0.00,0.000000,0.012821,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.012821,0.038462,0.012821,0.000000,0.00,0.000000,0.000000,0.064103,0.128205,0.000000,0.012821,0.00,0.000000,0.000000,0.025641,0.000000,0.000000,0.00,0.012821,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.012821,0.00,0.000000,0.00,0.012821,0.012821,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.012821,0.012821,0.025641,0.000000,0.000000,0.000000,0.00,0.000000,0.038462,0.000000,0.012821,0.00,0.012821,0.012821,0.012821,0.012821,0.025641,0.000000,0.00,0.025641,0.000000,0.000000,0.000000,0.012821,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
1,"► Alwal‎ (1 C, 1 F)",0.250000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.250000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.250000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,

In [28]:
len(kl_grouped[kl_grouped["Restaurant"] > 0])

27

Create a new DataFrame for Restaurant  data only

In [29]:

kl_mall = kl_grouped[["Neighborhoods","Restaurant"]]

In [30]:

kl_mall.head()

,Neighborhoods,Restaurant
0,"► Abids‎ (1 C, 13 F)",0.038462
1,"► Alwal‎ (1 C, 1 F)",0.000000
2,"► Ameerpet, Hyderabad‎ (3 C, 21 F)",0.010000
3,"► Bandlaguda, Rangareddy‎ (1 C, 2 F)",0.000000
4,"► Banjara Hills‎ (3 C, 25 F)",0.020000



7. Cluster Neighborhoods

Run k-means to cluster the neighborhoods in Hyderabad into 5 clusters.

In [31]:
# set number of clusters
kclusters = 3

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 1, 1, 1, 2, 2, 1, 1, 1, 0], dtype=int32)

In [32]:

# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [33]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Restaurant,Cluster Labels
0,"► Abids‎ (1 C, 13 F)",0.038462,2
1,"► Alwal‎ (1 C, 1 F)",0.000000,1
2,"► Ameerpet, Hyderabad‎ (3 C, 21 F)",0.010000,1
3,"► Bandlaguda, Rangareddy‎ (1 C, 2 F)",0.000000,1
4,"► Banjara Hills‎ (3 C, 25 F)",0.020000,2


In [34]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(52, 5)


,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
0,"► Abids‎ (1 C, 13 F)",0.038462,2,17.38980,78.47658
1,"► Alwal‎ (1 C, 1 F)",0.000000,1,17.53543,78.54427
2,"► Ameerpet, Hyderabad‎ (3 C, 21 F)",0.010000,1,17.43482,78.44949
3,"► Bandlaguda, Rangareddy‎ (1 C, 2 F)",0.000000,1,17.29982,78.46495
4,"► Banjara Hills‎ (3 C, 25 F)",0.020000,2,17.41535,78.43435


In [35]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(52, 5)


,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
44,► Sanathnagar‎ (8 F),0.066667,0,17.458760,78.443100
18,"► HITEC City‎ (5 C, 29 F)",0.070000,0,17.448230,78.374290
17,"► Golconda‎ (5 C, 4 F)",0.071429,0,17.389410,78.404060
50,"► Tarnaka‎ (1 C, 6 F)",0.055556,0,17.408935,78.326740
48,"► Sitaphalmandi‎ (1 C, 1 F)",0.055556,0,17.408935,78.326740
31,► Manikonda‎ (8 F),0.100000,0,17.401390,78.391630
13,► Domalguda‎ (3 C),0.048780,0,17.409950,78.482290
9,"► Cavalry Barracks, Hyderabad‎ (1 C)",0.055556,0,17.408935,78.326740
28,"► Madhapur‎ (1 C, 19 F)",0.081395,0,17.459000,78.368100
11,► Dabirpura‎ (1 C),0.055556,0,17.408935,78.326740


In [36]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [37]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

8. Examine Clusters

Cluster 0

In [38]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
44,► Sanathnagar‎ (8 F),0.066667,0,17.458760,78.44310
18,"► HITEC City‎ (5 C, 29 F)",0.070000,0,17.448230,78.37429
17,"► Golconda‎ (5 C, 4 F)",0.071429,0,17.389410,78.40406
50,"► Tarnaka‎ (1 C, 6 F)",0.055556,0,17.408935,78.32674
48,"► Sitaphalmandi‎ (1 C, 1 F)",0.055556,0,17.408935,78.32674
31,► Manikonda‎ (8 F),0.100000,0,17.401390,78.39163
13,► Domalguda‎ (3 C),0.048780,0,17.409950,78.48229
9,"► Cavalry Barracks, Hyderabad‎ (1 C)",0.055556,0,17.408935,78.32674
28,"► Madhapur‎ (1 C, 19 F)",0.081395,0,17.459000,78.36810
11,► Dabirpura‎ (1 C),0.055556,0,17.408935,78.32674


Cluster 1

In [39]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
26,► Kukatpally‎ (16 F),0.00,1,17.487350,78.420870
27,► L. B. Nagar‎ (16 F),0.00,1,17.512650,78.441290
19,"► Hayathnagar‎ (1 C, 14 F)",0.00,1,17.327070,78.605330
30,"► Malkajgiri‎ (3 C, 6 F)",0.00,1,17.439300,78.529200
41,"► Nizampet‎ (2 C, 32 F)",0.00,1,17.518330,78.381860
34,► Miyapur‎ (5 F),0.00,1,17.421020,78.582440
35,► Moazzam Jahi Market‎ (16 F),0.00,1,17.384480,78.474420
36,"► Moula-Ali‎ (3 C, 5 F)",0.00,1,17.465770,78.560180
37,"► Nacharam‎ (1 C, 4 F)",0.00,1,17.433510,78.566730
43,► Pedda Amberpet‎ (1 F),0.00,1,17.321150,78.642370


Cluster 2

In [40]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
40,"► Narayanguda‎ (1 C, 4 F)",0.019608,2,17.395474,78.497594
49,► Somajiguda‎ (5 F),0.020000,2,17.420720,78.463000
42,"► Old City (Hyderabad, India)‎ (8 C, 26 F)",0.032258,2,17.394870,78.470760
0,"► Abids‎ (1 C, 13 F)",0.038462,2,17.389800,78.476580
25,"► Koti, Hyderabad‎ (3 C, 7 F)",0.014706,2,17.385940,78.483380
32,► Masab Tank‎ (4 F),0.020000,2,17.400930,78.453620
24,"► Khairtabad‎ (1 C, 2 F)",0.030000,2,17.405920,78.458560
23,"► Kachiguda‎ (1 C, 4 F)",0.020833,2,17.386880,78.495530
22,"► Jubilee Hills‎ (3 C, 8 F)",0.030000,2,17.428650,78.397620
20,► Hyderguda‎ (2 F),0.032258,2,17.399230,78.480730
